# Statistics Primer

## Introduction

In this session we're going to grapple with two new challenges: (1) programming in Python and (2) statistics. Thankfully (1) is extremely straightforward because you will just be *running* code, rather than writing it. (2) Is where we will need to focus our efforts. It is worth this dedication, though, because without a good understanding of the relevant statistical concepts, understanding of extreme events (which are the critical drivers of *risk*) is difficult to say the least. 

By the end of this session you should have an understanding of: 

[1] Percentiles, exceedance probability, and return periods

[2] Non-parametric and parametric analyses 

... Which in turn lays the foundation for next week when we start looking at climate extremes in particular (and asking, for example, how we can model the changing *frequency* of dangerous heat events under climate warming). 

We will address these learning objectives by following this "Jupyter Notebook": an interactive story, with normal text in some places (like in this 'cell'), and Python computer code elsewhere (like below). To make the most of this resource, you must indeed *treat it like a story* -- read everything in the order it's presented, and run the code when told to do so. It is fine to work with one of your coursemates to discuss the notebook as you progress. I also encourage you to ask me (Tom) questions if you get stuck or want to know more. 


Let's get started, then, by running the code below. It sets up everything we need to begin analysis (and prints a congratulations message to screen, so that you know everything worked). 

**You 'run' a cell by either presssing the 'play' button at the top of the browser window (the button looks like a triangle on its side); or you can press ctrl + shift + enter on your keyboard (after clicking in the target cell**. 

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
from math import *
import warnings
from scipy import stats
warnings.filterwarnings("ignore")
%matplotlib inline
print("Well done. You just executed some Python code!")

## Non-parametric frequentist analysis

We're going to begin by using non-parametric analysis to analyse the heights of 1,000 women. Note that the precise definition of "non-parametric" is not terribly clear [see here for the Wikipedia explanation](https://en.wikipedia.org/wiki/Nonparametric_statistics), but for our purposes it means "making no assumptions at all about the *distribution* of the dataset" we are assessing. We will do nothing more than rank and count our data in our assessments of probability. 

We are going to use the heights dataset throughout this notebook -- mainly because we all understand height as a variable, so it's a good place to start. The *principles* we address, though, apply equally to the assessments of climate data that will be the focus of future sessions!

Run the code cell below to load the heights dataset into our notebook. The code computes the mean (the sum of all the heights, divided by 1,0000), and the [*standard deviation*](https://en.wikipedia.org/wiki/Standard_deviation) (a measure of variability around the mean), and prints them to scren. 

In [ ]:
heights=np.loadtxt("Data/Heights.csv") # Read in the sample of women's height
mean=np.mean(heights)
sigma=np.std(heights)
print("Mean is %.2f cm, standard deviation is %.2f cm"%(mean,sigma))

You should have seen a height of just over 160 cm, and a standard deviation of just under 10 cm. With those descriptive statistics out the way, let's examine how the data are *distributed* -- how common heights of different magnitudes are. Run the code below for a look. 

In [ ]:
# This code generates a histogram with 20 bins (categories)
fig,ax=plt.subplots(1,1)
out=ax.hist(heights,bins=20,facecolor='white',edgecolor='black')
ax.set_xlabel("Height (cm)")
ax.set_ylabel("Frequency (count)")

The chart above should be familiar: it is a histogram, and indicates how many women have heights between the values marked by the edges of the vertical bars. A more common application for this type of "frequentist" analysis, though, is to ask how many -- or what *proportion* of observations (in this case, heights) -- fall above or below a given value. 

For example, imagine you were designing a chair for female scientists, whose dimensions should suit their height. Because of engineering costs, it is decided that the chair should suit at least 98 % of women. (A specialist design will be developed for the shortest and tallest 1 %). 

To set the design standard, then, we therefore need to find the height that only 1 % of women are shorter than (the *lower* bound); *and* the height that only 1 % of women are taller than (the *upper* bound). Enter the **"percentile"** -- the value that 100-k % of the data equal or exceed (where k is a value between 0 and 100). See below for how we compute the lower and upper bounds using python. 

In [ ]:
lower=np.percentile(heights,1) # k is 1; We are finding the "1st percentile"
upper=np.percentile(heights,99) # k is 99; We are finding the "99th percentile"
print("Lower bound is %.1f cm\nUpper bound is %.1f cm"%(lower, upper))

So, if there are 500,000 female scientists in the UK, how many will be excluded by the lower/upper bounds of our design standard? 

The answer to this should be quite obvious after a moment's pause. We have, by definition, excluded 2 % of the population with our design standard (i.e., 1 in 50 women). 2 % of 500,000 is 10,000: this is how many scientists would need a custom chair. 

Why am I labouring this relatively obvious point? Because this general approach -- where we have a *sample* of data and must deduce some quantitiy relating to the *entire population* -- is common. Note that here, our *sample* is 1,000 strong, and the population comprises 500,000 individuals. We used the sample to compute the percentiles and then used that (which we implicitly assume represents *all* female scientists) to infer something about that wider population. 

Although this example deals with heights, I hope you can see how the same approach could be used to assess the threshold to identify, say, temperatures that are so hot they only occur on average once every 50 years (i.e., the 98 th percentile in a dataset of annual maximum temperatures) -- or indeed thresholds in any other meteorological quantity. 

Now imagine the chair designing committe come back to us and say that they were too hasty. They do not want to exclude 2 women in every hundred (or 1 in 50), and instead want to exclude only 1 in every 2,000. To make things a bit simpler, design innovations mean the chair is only limted in the upper-range of heights it can fit; we therefore only need to identify a new upper bound to inform the chair's design (the height equalled or *exceeded* by 1 in 2,000 women).  

Note an important point here: the language "1 in x" (meaning the value exceeded, on average, once every $x$ trials) is used all the time in climate science through the language of **return periods**. We *'return'* to this later when presenting the *CDF*. 

To be clear on these issues of syntax at this stage, though: a value equalled or exceeded 1 in 2,000 trials has a probability of being equalled or exceeded of 1/2,000 = 0.0005 (or 0.05 %) on any one individual trial; there is consequently a 100-0.05 = 99.95 % chance of *not* being equalled/exceeded on any trial (the *"non-exceedance probability"*). The value that corresponds to this rarity is the 99.95th percentile and has a *return period* of 1/0.0005 = 2,000 trials. "Trials" means individuals in the context of heights we're working with here; but it often means a unit of time in climate science (for example, days or years).  

Back to heights. Can we use the percentile function to obtain the threshold that only 1-in-2,000 female scientists exceed?

In [ ]:
percentile=100-(1/2000*100)
print("Finding the %.2f th percentile..."%percentile)
upper=np.percentile(heights,percentile) 
print("... The answer is %.1f cm"%upper)

Certainly! 

... Although, all this shows is that python does not return an error. Does the answer make any sense? 

Perhaps. But you may have noticed a potential problem with what we're doing here. Run the code below to find the tallest person in our dataset. 

In [ ]:
print("The maximum value in the dataset (that constitutes the heights of %.0f individuals) is %.1f cm" % \
      (len(heights),heights.max()))

The value returned is the maximum of 1,000 members (by definition, the 1-in-1,000 individual), yet our estimate of the 1-in-2,000 height is lower! 

The above highlights an important limitation of **non-parametric** analysis: it cannot be used to tell us anything about cases beyond the range of our measurements. In this case, that means that we don't know what the "1-in-2,000" individual looks like. **Parametric** methods are not so limited: they make assumptions about the data generating processes, enabling us to extrapolate beyong the limits of our data sample. 

## Parametric frequentist analysis

In parametric analysis we make an assumption about the *probability distribution* (i.e., the relationship between the magnitude of values and the frequency with which they occur) that our data have been drawn from. One of the most common data distributions is the "Normal" distribution (also called the "Gaussian"). This "bell-shaped" distribution appears very often (hence the name) because it is readily produced by combining random features additively. The *probability distribution function* of the Normal distribution is given by:

\begin{aligned}
p(x)=\frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}
\end{aligned}

Whilst this may look complicated, it isn't. The function will simply return a value (always less than 1) depending on the value of $x$, and the distribution parameters: $\sigma$ (the standard deviation), and $\mu$ (the mean). Note that the other terms you may not recognise are [$\pi$](https://en.wikipedia.org/wiki/Pi) and $e$ is [Euler's number](https://en.wikipedia.org/wiki/E_(mathematical_constant) -- they are "constants". 

In the plot below we evaluate $p(x)$ for all values of $x$ (between 100 and 220 cm) using the equation above. We then plot this on the same histogram as earlier, which has now been scaled so that that the bars sum to 1. This enables a fair comparison to the probability density function defined above. Run the code below to generate the plot. 

In [ ]:
x=np.linspace(120,205,100) # set up values of x (heights) to query 
sigma=10
mean=161.6
px=1/(sigma*np.sqrt(2*np.pi))*np.exp(-1/2.*((x-mean)/sigma)**2)
fig,ax=plt.subplots(1,1)
ax.plot(x,px,color='red')
out=ax.hist(heights,20,density=True,facecolor='white',edgecolor='black')
ax.set_xlabel("Height (cm)")
ax.set_ylabel("p(x)")

The red line matches the histogram well! This suggests (as we might expect) that the heights are drawn from a Normal distribution. 

In the study of extremes, is is convenient to query the probability distribution to find a given percentile. We can do that with the help of the **Cumulative Distribution Function (CDF)**. For the Normal distribution this is defined: 

\begin{aligned}
d(x)=\frac{1}{2}[erf(\frac{x}{\sqrt(2)})+1]
\end{aligned}

Where $x$ has been transformed into a ["standard normal distribution"](https://mathworld.wolfram.com/StandardNormalDistribution.html) -- i.e., with a mean of 0 and a standard deviation of 1, by subtracting the mean of $x$ ($\mu$) from each value, and then dividing by the standard deviation ($\sigma$). Note that $erf$ is the [error function](https://en.wikipedia.org/wiki/Error_function). 

To be clear, the equation above returns the probability that $x$ will *not* be exceeded. Accordingly, the *CDF* will only ever return values between 0-1 (i.e., between 0-100 %). Note that this is essentially the *inverse* (opposite) of the percentile, but in decimal form. That is, if $d(175 cm)$ = 0.9, the 90th percentile for the population would be 175 cm. 

Below, we evaluate and plot $d(x)$ for a prescribed range of heights, given a mean of $\mu$=161.04 cm and standard deviation of $\sigma$=9.66 cm. 

We also plot the empirical non-exceedance probability, simply defined as the fraction of the 1,000 member sample with heights below the same points in the arbitrary list. The code for this is included below, but there's no need to "learn it"; you can just observe/compare the empirical (red) and theroetical (blue) non-exceedance probabilities in the plot. 

In [ ]:
x_standard=(x-mean)/sigma # Convert to standard normal
dx=np.array([1/2.*(erf(i/np.sqrt(2))+1)*100 for i in x_standard]) # compute d(x)
empirical_dx=np.array([100-np.sum(heights>i)/np.float(len(heights))*100 for i in heights]) # empirical d(x)
# Plot
fig,ax=plt.subplots(1,1)
ax.plot(x,dx)
ax.plot(heights[np.argsort(heights)],empirical_dx[np.argsort(heights)],color='red',linestyle='--')
ax.grid()
ax.set_xlabel("Height (cm)")
ax.set_ylabel("d(x) [%]")

### Parametric distributions provide a "smoothed" estimate 

If you're having trouble spotting the difference between the red and blue curves, that's no surpise. As with the comparison between $p(x)$ and the histogram, we note that the match is very close because our sample of heights is quite large (1,000 members) and is drawn from a normal distribution. Had our sample been smaller, it is more likely that our empirical estimate of non-exceedance probability would look a little different -- even if drawn from the same normal distribution. Why? Well, broadly speaking, the [law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers) indicates that probabilities converge on their "expected" values with a sufficiently large number of trials. In this case, the number of trials is the sample size of heights. 

To illustrate the effect of decreasing the sample size, run the code below. 

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(x,dx,color="black",linewidth=3)
for size in [5,10,15,20,25,30]:
    sample=np.random.normal(loc=mean,scale=sigma,size=size)
    empirical_dx=np.array([100-np.sum(sample>i)/np.float(len(sample))*100 for i in sample])
    ax.plot(sample[np.argsort(sample)],empirical_dx[np.argsort(sample)],label="%.0f"%size)
ax.grid()
ax.set_xlabel("Height (cm)")
ax.set_ylabel("d(x) [%]")   
ax.legend()

As should be clear from the above, the larger sample size, the closer the empirical estimate to the population $d(x)$. 

So, there are two reasons why we might want to use a probability distribution to assess frequency (or "rarity") rather than an empirical estimate: 

(1) It enables us to extrapolate beyond the bounds of our observation;

(2) It smoothes out batch-to-batch sampling variation (most problematic with small sample sizes). 

There are, however, considerable challenges to using a probability distribution. **First**, we must select the *right* distribution. In the above this was the *normal* distribution, but *many* others exist, and we must decide which (we think) describes the population we are dealing with. **Second**, we must identify the values for the paramaters that describe the given probability distribution (i.e., its shape). In the case of the normal distribution, we need to quantify two paramaters: the mean and standard deviation; but some other distributions require additional paramaters to be set. Both of these "challenges" exist because we must *estimate* both the distribution *and* its paramaters from a finite sample. As I hope is intuitive from the example above, the larger the sample, the closer we should get to the right answer. However, we can never be sure we have the "right" probability distribution and paramaters for the population -- only that statisticallly it is *unlikely* that we are wrong. We will discuss this more as the course progresses. 


## The *CDF* and 'rarity'

Back to our aim of finding the 1-in-2,000 height. How can we use the normal *CDF* to find out what the threshold is? Well, recall that the CDF returns the probability of *not* equaling/exceeding $x$ (i.e., a given height). What we want, then, is the *inverse* of that function -- i.e., which value of $x$ is *not* exceeded 99.95 % of the time? (i.e. what is the 99.95th percentile according to the Normal distribution?)

Mathematically, we will rely on functions already available in the Python programming language (or provided by me) to do such inversions. At this stage it's just important to appreciate the concept of what we're doing. The code below -- and the plot it produces -- should help with this. 

We will compute $d(x)$ (the *CDF*), then turn this into the exeedance probability (1-$d(x)$), and covert it to a *return period* by $1/(1-d(x)$). Once we plot this, the 99.95th percentile is equal to the 1-in-2,000 return period (i.e., the height exceeded by 1-in-2,000 people according to the Normal distribution with 𝜇 = 161.04 cm and 𝜎 = 9.66 cm). The red lines mark this return period on the x-axis, and where it intercepts the normal curve on the y-axis.

In [ ]:
pex=1-np.array([1/2.*(erf(i/np.sqrt(2))+1) for i in x_standard])
return_periods=1/pex
crit=1-1/2000.
inv=(stats.norm.ppf(crit)*sigma+mean)
fig,ax=plt.subplots(1,1)
ax.plot(return_periods,x)
return_periods
ax.set_xscale('log')
ax.axvline(1/(1/2000.),color='red',linestyle='--')
ax.axhline(inv,color='red',linestyle='--')
ax.grid()
ax.set_xlabel("Return period (i.e., 1-in-x)")
ax.set_ylabel("Height (cm)")

The interception point on the y-axis is our 1-in-2,000 height. Reading off the curve, we see that this is 194.5 cm! The term "return periods" is inappropriate for discussing people's height, but it serves to illustrate the key concepts we will be using in the following sessions.  

## Beyond the normal distribution 
Finally, note that whilst we have illustrated parametric frequency analysis with the Normal distribution, (many) others exist. To appreciate this diversity of probability distributions, run the code below to plot $p(x)$ for a range of commonly used distributions. Notice how well many of them appear to fit our sample! (The "right" distribution is the normal distribuion -- top left). A longer list of distributions availble in one of Python's statistics modules is included [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/stats.html). 

This concludes the directed section of this workshop. You should then proceed to the **Questions** section for your home/self-guided working. 

In [ ]:
dists=["norm","gamma","skewnorm","cosine","gumbel_r","gumbel_l","genlogistic","laplace","pearson3","t",
      "tukeylambda","gausshyper"]
p=stats.expon.pdf(x)
fig,ax=plt.subplots(3,4)
fig.set_size_inches(15,10)
for i in range(len(dists)):
    d=getattr(stats,dists[i])
    params=d.fit(heights)
    if len(params)==2:
        p=d.pdf(x,params[0],params[1])
    elif len(params)==3:
        p=d.pdf(x,params[0],params[1],params[2])
    out=ax.flat[i].hist(heights,20,density=True,facecolor='white',edgecolor='black')
    ax.flat[i].plot(x,p)
    ax.flat[i].set_title(dists[i])
plt.tight_layout()

## Self working

Please answer/complete the following. We will go through the answers in the next session, so it is essential that you complete them ahead of time. 

[1] A lottery machine holds 59 coloured balls, each labelled with a different number (from 1-59). What is the probability that the first number to emerge is greater than 50?

[2] How many selections (trials) from the lottery machine should you expect to make until you choose a number less than 5?

[3] Imagine sampling from the lottery machine many, many times. Each time you make a note of the number, replace the ball, before drawing again on the next go. At the end of your mammoth sampling effort, you have a list of numbers that constitutes your dataset. Sketch how you think a histogrom of these data would look. 

[4] Repeat the thought experiment in [3], but this time imagine that in each sample you selected 6 balls and added them up and wrote down this sum. How does the histogram of this dataset look?

[5] Skim the papers listed below and, for each, identify:
- (a) which meterorological quantity do the authors assess? 
- (b) Which probability distribution is applied to assess the rarity of the events? 

Papers: 
- [Matthews et al. (2016)](https://www.sciencedirect.com/science/article/pii/S221209631600005X) 
- [Schar et al. (2004)](https://www.nature.com/articles/nature02300) 
- [Matthews et al.(2018)](https://iopscience.iop.org/article/10.1088/1748-9326/aa98c8)


